# Loading

In [16]:
#Here, all the necessary imports are made.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import display

from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import csv

In [17]:
#Here, the data is loaded, adjust path as needed.
DATA_DIR = '/Users/larsheijnen/Thesis/data/motor/MDS-UPDRS_Part_III_21Mar2025.csv'
data = pd.read_csv(DATA_DIR)

In [ ]:
# Print the number of features (columns) in the dataset
print("Number of features:", len(data.columns))

# Display the column names
print("\nFeature names:")
for col in data.columns:
    print(f"- {col}")

unique_patients = data['PATNO'].nunique()
print(f"Number of unique patients: {unique_patients}")

# Display some basic statistics about patients
print("\nPatient visit statistics:")
patient_visits = data.groupby('PATNO').size()
print(f"Average visits per patient: {patient_visits.mean():.2f}")
print(f"Min visits per patient: {patient_visits.min()}")
print(f"Max visits per patient: {patient_visits.max()}")

In [ ]:
# Convert date columns to datetime where possible
for col in ['INFODT', 'EXAMDT', 'ORIG_ENTRY', 'LAST_UPDATE']:
    if col in data.columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')

# Convert specific date columns
data['EXAMDT'] = pd.to_datetime(data['EXAMDT'], errors='coerce')
data['INFODT'] = pd.to_datetime(data['INFODT'], errors='coerce')

# Fallback: fill missing EXAMDT with INFODT
data['EXAMDT'] = data['EXAMDT'].fillna(data['INFODT'])

print("Earliest visit:", data['EXAMDT'].min())
print("Latest visit:", data['EXAMDT'].max())
print("Median interval between visits (days):", 
      data.sort_values(['PATNO', 'EXAMDT']).groupby('PATNO')['EXAMDT'].diff().median())


#Check outliers or wrong values
np3_cols = [col for col in data.columns if col.startswith('NP3') and col != 'NP3TOT']

# Find out-of-range values
out_of_range = (data[np3_cols] < 0) | (data[np3_cols] > 4)
if out_of_range.any().any():
    print("Warning: Out-of-range NP3 scores detected!")
    display(data.loc[out_of_range.any(axis=1), ['PATNO', 'EXAMDT'] + np3_cols])
else:
    print("All NP3 scores within expected range (0–4). Ignored values in the NP3TOT column.")

In [ ]:
# Replace 101 values with NaN in NP3 columns, 101 values are placed in the data to indicate missing values.
np3_cols = [col for col in data.columns if col.startswith('NP3') and col != 'NP3TOT']
data[np3_cols] = data[np3_cols].replace(101, np.nan)
# Also replace 101 in NHY column with NaN
data['NHY'] = data['NHY'].replace(101, np.nan)


# Find out-of-range values
# Find and remove out-of-range values
out_of_range = (data[np3_cols] < 0) | (data[np3_cols] > 4)
if out_of_range.any().any():
    print("Warning: Out-of-range NP3 scores detected!")
    print(f"Found {out_of_range.sum().sum()} out-of-range values")
    # Replace out-of-range values with NaN
    data.loc[out_of_range] = np.nan
    print("Out-of-range values have been replaced with NaN")
else:
    print("All NP3 scores within expected range (0–4). Ignored NP3TOT.")

print(f"Remaining rows after filtering NP3TOT: {len(data)}")

# Drop rows where NP3TOT is NaN
data = data.dropna(subset=['NP3TOT'])
print(f"Remaining rows after dropping NaN NP3TOT: {len(data)}")


In [ ]:
# Count unique patients
unique_patients = data['PATNO'].nunique()
print(f"Number of unique patients: {unique_patients}")

# Display some basic statistics about patients
print("\nPatient visit statistics:")
patient_visits = data.groupby('PATNO').size()
print(f"Average visits per patient: {patient_visits.mean():.2f}")
print(f"Min visits per patient: {patient_visits.min()}")
print(f"Max visits per patient: {patient_visits.max()}")


# Exploratory Data Analysis / # Data Analysis

Visualize and quantify missingness across key variables using heatmaps and summary statistics.

Summarize the dataset structure, check column meanings, and display initial rows to understand the data.

In [ ]:
# Sort by PATNO and visit date (prefer EXAMDT, fallback to INFODT if EXAMDT is missing)
#INFODT is the date of the information, EXAMDT is the date of the visit. They are most of the time the same.
if 'EXAMDT' in data.columns:
    data = data.sort_values(['PATNO', 'EXAMDT'])
elif 'INFODT' in data.columns:
    data = data.sort_values(['PATNO', 'INFODT'])

# Remove post-baseline visits with EXAMDT on or before baseline for each patient. This unlikely ever happens.
if 'EXAMDT' in data.columns and 'PATNO' in data.columns:
    baseline_dates = data.groupby('PATNO')['EXAMDT'].transform('min')
    # Keep only visits after baseline or the baseline itself
    mask = (data['EXAMDT'] > baseline_dates) | (data['EXAMDT'] == baseline_dates)
    n_before = data.shape[0]
    data = data[mask]
    n_after = data.shape[0]
    print(f"Removed {n_before - n_after} visits with EXAMDT on or before baseline (except baseline itself).")

assert pd.api.types.is_datetime64_any_dtype(data['EXAMDT'])

# Create a visit number per patient
data['VISIT_NUM'] = data.groupby('PATNO').cumcount()

# Show a summary of the data.
print(data[['PATNO', 'EXAMDT', 'EVENT_ID', 'PDSTATE', 'NP3TOT', 'VISIT_NUM']].head(10))

In [ ]:
# Check for paired ON/OFF visits (same PATNO, EVENT_ID, different PDSTATE).
paired = data.groupby(['PATNO', 'EVENT_ID'])['PDSTATE'].nunique()
paired_onoff = paired[paired > 1]
print("Number of ON/OFF paired visits:", paired_onoff.count())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Descriptive statistics for NP3TOT, NHY, and key NP3 items
key_items = ['NP3TOT', 'NHY', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3GAIT', 'NP3BRADY']
desc = data[key_items].describe()
print("Descriptive statistics:\n", desc)

# Plot: Distribution of NP3TOT
plt.figure(figsize=(8, 4))
sns.histplot(data['NP3TOT'].dropna(), bins=30, kde=True)
plt.title('Distribution of NP3TOT')
plt.xlabel('NP3TOT')
plt.ylabel('Count')
plt.show()

# Plot: Correlation heatmap for key NP3 items
corr_matrix = data[key_items].corr()
plt.figure(figsize=(7, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix: Key NP3 Items')
plt.show()

In [ ]:
# Plot the correlation matrix for all motor-related features as a heatmap,
# but ignore any features related to NP3TOT log or sqrt transformations.

# Define all motor-related features (based on MDS-UPDRS Part III, NP3* columns)
# Exclude NP3TOT_log and NP3TOT_sqrt if present
motor_features = [
    col for col in data.columns
    if col.startswith('NP3') and col not in ['NP3TOT_log', 'NP3TOT_sqrt']
]

# Compute correlation matrix for these features
motor_corr = data[motor_features].corr()

# Plot the correlation heatmap with improved formatting
plt.figure(figsize=(18, 14))  # Make the figure larger for better readability
sns.heatmap(
    motor_corr, 
    annot=True, 
    fmt=".2f", 
    cmap='coolwarm', 
    vmin=-1, vmax=1, 
    annot_kws={"size":8},   # Smaller font for annotations
    cbar_kws={"shrink": 0.8, "aspect": 30}
)
plt.title('Correlation Matrix: All Motor-Related Features (NP3*)', fontsize=16)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# Check visit intervals (in days) for a sample patient, this is important for later on.
sample_patno = data['PATNO'].dropna().unique()[0]
sample = data[data['PATNO'] == sample_patno].sort_values('EXAMDT')
if sample['EXAMDT'].notna().sum() > 1:
    sample['VISIT_DIFF_DAYS'] = sample['EXAMDT'].diff().dt.days
    print(sample[['PATNO', 'EXAMDT', 'VISIT_DIFF_DAYS']])

In [ ]:
from scipy.stats import shapiro
import scipy.stats as stats
import matplotlib.pyplot as plt

# Q–Q plot for NP3TOT
# The Q–Q plot is used to check if the data is normally distributed.
plt.figure(figsize=(6, 6))
stats.probplot(data['NP3TOT'].dropna(), dist="norm", plot=plt)
plt.title('Q–Q Plot of NP3TOT')
plt.show()

# Shapiro–Wilk test
stat, p = shapiro(data['NP3TOT'].dropna())
print(f"Shapiro–Wilk test for NP3TOT: W={stat:.3f}, p={p:.3g}")
if p < 0.05:
    print("NP3TOT is NOT normally distributed (reject H0 at α=0.05).")
else:
    print("NP3TOT is consistent with normality (fail to reject H0 at α=0.05).")


#The distribution of "NP3TOT" is not normal. It appears to be positively skewed, with a likely many low values.
# It is important to note this, especially when performing time-series modelling, since many models assume normality.
# Rejecting the null hypothesis, the data is not normally distributed.


In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

# Transformation: Shift to avoid log(0)
data['NP3TOT_log'] = np.log1p(data['NP3TOT'])  # log(1 + x)
data['NP3TOT_sqrt'] = np.sqrt(data['NP3TOT'])

# Check normality again
for col in ['NP3TOT', 'NP3TOT_log', 'NP3TOT_sqrt']:
    stat, p = stats.shapiro(data[col].dropna())
    print(f"{col}: Shapiro-Wilk p-value = {p:.5f}")

    stats.probplot(data[col].dropna(), dist="norm", plot=plt)
    plt.title(f"Q–Q plot: {col}")
    plt.show()


# Log Transformed (NP3TOT_log):
# Q-Q Plot: Some improvement, especially in tail behavior, but still clear deviation.
# Shapiro-Wilk p-value: 0.00000 → Still not normal.
# Observation: The stair-step effect suggests many repeated small values (possibly zeros).

# Square Root Transformed (NP3TOT_sqrt):
# Q-Q Plot: Slightly better fit in mid-range values, but tails still deviate.
# Shapiro-Wilk p-value: 0.00000 → Still not normal.

# To conclude:
# Neither log nor square root transformation fully normalize the distribution.
# The data likely contains a large number of zeros or near-zero values (zero-inflated), and is intrinsically non-normal.

#Modeling Perspective:
# Use models that do not assume normality, like:
# Generalized Linear Models (GLMs) with Poisson or Negative Binomial link (if count-like).

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data['VISIT_NUM'], bins=20, kde=True)
plt.title('Distribution of Visit Numbers')
plt.xlabel('Visit Number')
plt.ylabel('Frequency')
plt.show()

In [30]:
missing_patients = {}
for col in key_items:
    n_missing = data.groupby('PATNO')[col].apply(lambda x: x.isna().any()).sum()
    pct_missing = 100 * n_missing / data['PATNO'].nunique()
    missing_patients[col] = pct_missing

In [ ]:
# Calculate mean and standard deviation of NP3TOT by visit number, up to visit 30
mean_by_visit = data.groupby('VISIT_NUM')['NP3TOT'].mean()
std_by_visit = data.groupby('VISIT_NUM')['NP3TOT'].std()

visit_mask = mean_by_visit.index <= 30
mean_by_visit_30 = mean_by_visit[visit_mask]
std_by_visit_30 = std_by_visit[visit_mask]

plt.figure(figsize=(8, 4))
plt.plot(mean_by_visit_30.index, mean_by_visit_30.values, marker='o', color='#2874A6', label='Mean NP3TOT')
plt.fill_between(
    mean_by_visit_30.index,
    mean_by_visit_30.values - std_by_visit_30.values,
    mean_by_visit_30.values + std_by_visit_30.values,
    color='#85C1E9', alpha=0.5, label='±1 SD'
)
plt.xlabel('Visit Number')
plt.ylabel('Mean NP3TOT')
plt.title('Mean NP3TOT by Visit Number (with SD, Visits 1–30)')
plt.ylim(bottom=0)
plt.xlim(left=0.5, right=30.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 2. Stratify by PDSTATE (ON/OFF)
print("\nNP3TOT by PDSTATE:")
print(data.groupby('PDSTATE')['NP3TOT'].describe())

plt.figure(figsize=(8, 6))
sns.boxplot(x='PDSTATE', y='NP3TOT', data=data)
plt.title('Boxplot of NP3TOT by PDSTATE (ON/OFF)')
plt.xlabel('PDSTATE')
plt.ylabel('NP3TOT')
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
sns.kdeplot(data=data, x='NP3TOT', hue='PDSTATE', fill=True, common_norm=False, alpha=0.5)
plt.title('Density Plot of NP3TOT by PDSTATE (ON/OFF)')
plt.xlabel('NP3TOT')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# NP3TOT scores are consistently higher in the OFF state, confirming that patients exhibit more severe symptoms when not medicated.
# This validates that PDSTATE is a strong stratifying factor, and any modeling should likely adjust for or include PDSTATE.
# The shift in all percentiles and the higher standard deviation in OFF state supports the clinical reality of fluctuating symptom severity with medication.

In [ ]:
# 3. Stratify by EVENT_ID (visit)
print("\nNP3TOT by EVENT_ID (first 5 visits):")
print(data.groupby('EVENT_ID')['NP3TOT'].describe().head())

#Variability is high across events, which aligns with Parkinson’s disease’s fluctuating nature.

In [ ]:
sample_patnos = [3451, 3203, 4057, 3808, 3448]

fig, axes = plt.subplots(1, 5, figsize=(18, 4), sharey=True)
for ax, patno in zip(axes, sample_patnos):
    patient = data[(data['PATNO'] == patno) & data['EXAMDT'].notna() & data['NP3TOT'].notna()]
    patient = patient.sort_values('EXAMDT')
    ax.plot(patient['EXAMDT'], patient['NP3TOT'], marker='o')
    ax.set_title(f'PATNO {patno}')
    ax.set_xlabel('EXAMDT')
axes[0].set_ylabel('NP3TOT')
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib.dates as mdates

plt.figure(figsize=(10, 6))
for patno in sample_patnos:
    patient = data[(data['PATNO'] == patno) & data['EXAMDT'].notna() & data['NP3TOT'].notna()]
    patient = patient.sort_values('EXAMDT')
    if len(patient) > 2:
        # Convert EXAMDT to matplotlib date numbers for LOWESS
        x = mdates.date2num(patient['EXAMDT'])
        y = patient['NP3TOT'].values
        smoothed = lowess(y, x, frac=0.5)
        # Plot original points
        plt.plot(patient['EXAMDT'], y, 'o', alpha=0.3)
        # Plot smoothed line, converting x back to datetime
        plt.plot(mdates.num2date(smoothed[:, 0]), smoothed[:, 1], label=f'PATNO {patno}')
plt.xlabel('EXAMDT')
plt.ylabel('NP3TOT')
plt.title('Smoothed NP3TOT over time')
plt.legend()
plt.show()

# Modeling Disease Progression

Fit linear mixed models or other longitudinal models to estimate progression rates and account for within-patient correlation.

In [38]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import pearsonr

In [ ]:
# 1. Calculate change in NP3TOT from baseline for each patient
data['NP3TOT_BL'] = data.groupby('PATNO')['NP3TOT'].transform('first')
data['NP3TOT_CHANGE'] = data['NP3TOT'] - data['NP3TOT_BL']
print("Change from baseline (first 10 rows):")
print(data[['PATNO', 'EXAMDT', 'NP3TOT', 'NP3TOT_BL', 'NP3TOT_CHANGE']].head(10))

#Allows tracking disease progression relative to baseline, rather than absolute scores.
# Helps in:
    # Longitudinal plotting (e.g., spaghetti plots or smoothed trends).
    # Modeling change as an outcome (e.g., mixed-effects models).
    # Identifying responders vs. non-responders to treatment/intervention.


In [ ]:
import numpy as np

# 2. Model trajectories: Linear mixed model (NP3TOT ~ time)
# Only use rows with valid EXAMDT and NP3TOT
model_data = data.dropna(subset=['EXAMDT', 'NP3TOT'])
model_data['DAYS_SINCE_BL'] = (model_data['EXAMDT'] - model_data.groupby('PATNO')['EXAMDT'].transform('min')).dt.days
md = smf.mixedlm("NP3TOT ~ DAYS_SINCE_BL", model_data, groups=model_data["PATNO"])
mdf = md.fit()
print(mdf.summary())

# Plot observed NP3TOT vs. DAYS_SINCE_BL with model prediction line
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.scatter(model_data['DAYS_SINCE_BL'], model_data['NP3TOT'], alpha=0.1, label='Observed', s=10)

# Predicted line (fixed effects only)
x_pred = np.linspace(model_data['DAYS_SINCE_BL'].min(), model_data['DAYS_SINCE_BL'].max(), 200)
y_pred = mdf.params['Intercept'] + mdf.params['DAYS_SINCE_BL'] * x_pred
plt.plot(x_pred, y_pred, color='red', linewidth=2, label='Model Prediction')

plt.xlabel('Days Since Baseline')
plt.ylabel('NP3TOT')
plt.title('Linear Mixed Model: NP3TOT vs. Time')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# — Cell: Build DAYS_SINCE_BL and verify PDSTATE —
import pandas as pd

# 1. Inspect what you’ve got
print("Columns in `data`:", data.columns.tolist())

# 2. Parse your exam-date (replace 'EXAMDT' if your date column is named differently)
data['EXAMDT'] = pd.to_datetime(data['EXAMDT'])

# 3. Compute each patient’s baseline date
data['baseline_date'] = data.groupby('PATNO')['EXAMDT'].transform('min')

# 4. Days since baseline
data['DAYS_SINCE_BL'] = (data['EXAMDT'] - data['baseline_date']).dt.days

# 5. Check your PD-state column
print("Unique PDSTATE values:", data['PDSTATE'].unique())

# 6. (Optional) If your column is named differently, rename it:
# data.rename(columns={'YourStateCol':'PDSTATE'}, inplace=True)

# 7. Now filter to OFF visits and drop any remaining NaNs
df = (
    data
    .query("PDSTATE == 'OFF'")
    .dropna(subset=['NP3TOT','DAYS_SINCE_BL','PATNO'])
    .assign(
        PATNO = lambda d: d['PATNO'].astype('category'),
        PDSTATE = lambda d: d['PDSTATE'].astype('category')
    )
)

print("Prepared df shape:", df.shape)
df.head()

In [ ]:
df = (
    data
    .dropna(subset=['NP3TOT', 'DAYS_SINCE_BL', 'PATNO', 'PDSTATE'])
    .assign(
        # categorical grouping & state
        PATNO        = lambda d: d['PATNO'].astype('category'),
        PDSTATE      = lambda d: d['PDSTATE'].astype('category'),
        # quadratic time
        DAYS_SINCE_BL2 = lambda d: d['DAYS_SINCE_BL'] ** 2
    )
)

print("Prepared df:", df.shape, "rows")

# 3. Fit GEE with Negative Binomial family
import statsmodels.formula.api as smf
from statsmodels.genmod.families import NegativeBinomial
from statsmodels.genmod.cov_struct import Exchangeable

formula = "NP3TOT ~ DAYS_SINCE_BL + DAYS_SINCE_BL2 + PDSTATE"

gee_pd = smf.gee(
    formula,
    groups=df["PATNO"],
    data=df,
    family=NegativeBinomial(),
    cov_struct=Exchangeable()
).fit()

print(gee_pd.summary())

In [ ]:
import matplotlib.pyplot as plt

# Pearson residuals and fitted means from gee_pd
resid = gee_pd.resid_pearson
fitted = gee_pd.fittedvalues

plt.scatter(fitted, resid, alpha=0.3)
plt.axhline(0, color='black', lw=1)
plt.xlabel("Fitted mean NP3TOT")
plt.ylabel("Pearson residual")
plt.title("Residuals vs. Fitted")
plt.show()

In [ ]:
import statsmodels.api as sm

# 2a. Fit an NB–GLM to estimate alpha
nb_glm = sm.GLM.from_formula(
    "NP3TOT ~ DAYS_SINCE_BL + DAYS_SINCE_BL2 + PDSTATE",
    data=df,
    family=sm.families.NegativeBinomial()
).fit()

print("Estimated α (dispersion) =", nb_glm.scale)

# 2b. Re-run GEE with that α
from statsmodels.genmod.families.family import NegativeBinomial as NBfam

fam = NBfam(alpha=nb_glm.scale)
gee_refit = smf.gee(
    "NP3TOT ~ DAYS_SINCE_BL + DAYS_SINCE_BL2 + PDSTATE",
    groups=df["PATNO"],
    data=df,
    family=fam,
    cov_struct=Exchangeable()
).fit()

print(gee_refit.summary())

In [ ]:
# 3. Treatment effect: ON vs. OFF within-patient (paired analysis)
paired_idx = data.groupby(['PATNO', 'EVENT_ID'])['PDSTATE'].transform('nunique') > 1
paired_data = data[paired_idx]
on_scores = paired_data[paired_data['PDSTATE'] == 'ON'].set_index(['PATNO', 'EVENT_ID'])
off_scores = paired_data[paired_data['PDSTATE'] == 'OFF'].set_index(['PATNO', 'EVENT_ID'])
common_idx = on_scores.index.intersection(off_scores.index)
np3tot_diff = on_scores.loc[common_idx, 'NP3TOT'] - off_scores.loc[common_idx, 'NP3TOT']
print("\nON - OFF NP3TOT difference (summary):")
print(np3tot_diff.describe())

In [ ]:
from scipy.stats import ttest_rel, wilcoxon

# Paired t-test
t, p_t = ttest_rel(on_scores['NP3TOT'], off_scores['NP3TOT'])
print(f"Paired t-test ON vs. OFF: t={t:.2f}, p={p_t:.3g}")

# Wilcoxon signed-rank test (non-parametric)
w, p_w = wilcoxon(on_scores['NP3TOT'], off_scores['NP3TOT'])
print(f"Wilcoxon signed-rank test ON vs. OFF: W={w:.2f}, p={p_w:.3g}")

if p_t < 0.05:
    print("Significant difference between ON and OFF (paired t-test).")
if p_w < 0.05:
    print("Significant difference between ON and OFF (Wilcoxon).")

In [ ]:
# 4. Compare treated vs. untreated over time (PDTRTMNT)
if 'PDTRTMNT' in data.columns:
    treated = data[data['PDTRTMNT'] == 1.0]
    untreated = data[data['PDTRTMNT'] == 0.0]
    print("\nMean NP3TOT by visit (treated):")
    print(treated.groupby('VISIT_NUM')['NP3TOT'].mean().head())
    print("\nMean NP3TOT by visit (untreated):")
    print(untreated.groupby('VISIT_NUM')['NP3TOT'].mean().head())

    # Calculate change from baseline for each patient
    data['NP3TOT_BL'] = data.groupby('PATNO')['NP3TOT'].transform('first')
    data['NP3TOT_CHANGE'] = data['NP3TOT'] - data['NP3TOT_BL']

    # Plot mean change from baseline by group
    plt.figure(figsize=(8, 4))
    treated_change = data[data['PDTRTMNT'] == 1.0].groupby('VISIT_NUM')['NP3TOT_CHANGE'].mean()
    untreated_change = data[data['PDTRTMNT'] == 0.0].groupby('VISIT_NUM')['NP3TOT_CHANGE'].mean()
    plt.plot(treated_change, marker='o', label='Treated')
    plt.plot(untreated_change, marker='o', label='Untreated')
    plt.xlabel('Visit Number')
    plt.ylabel('Mean Change in NP3TOT from Baseline')
    plt.title('Mean Change in NP3TOT by Visit: Treated vs Untreated')
    plt.legend()
    plt.show()

In [ ]:
# 5. Progression rates: Annualized change in NP3TOT, NHY
def annualized_change(df, score):
    df = df.sort_values('EXAMDT')
    if len(df) < 2:
        return np.nan
    days = (df['EXAMDT'].iloc[-1] - df['EXAMDT'].iloc[0]).days
    if days == 0:
        return np.nan
    return (df[score].iloc[-1] - df[score].iloc[0]) / (days / 365.25)

annual_np3tot = data.dropna(subset=['EXAMDT', 'NP3TOT']).groupby('PATNO').apply(annualized_change, 'NP3TOT')
print("\nAnnualized NP3TOT change (summary):")
print(annual_np3tot.describe())

In [ ]:
plt.figure(figsize=(10, 5))
data['PDSTATE'] = data['PDSTATE'].astype('category')
for state in data['PDSTATE'].cat.categories:
    state_data = data[data['PDSTATE'] == state]
    mean_trend = state_data.groupby('VISIT_NUM')['NP3TOT'].mean()
    plt.plot(mean_trend.index, mean_trend.values, marker='o', label=f'{state} Mean')
plt.xlabel('Visit Number')
plt.ylabel('Mean NP3TOT')
plt.title('Mean NP3TOT by Visit Number and PDSTATE')
plt.legend()
plt.show()

# 1. Clear Separation Between ON and OFF States
# -------------------------------------------
# • OFF state (blue): Higher NP3TOT scores consistently across all visits
# • ON state (orange): Lower NP3TOT scores throughout, indicating symptom improvement with medication

# 2. Temporal Trends
# ----------------
# • Both groups show a gradual increase in NP3TOT over time, reflecting disease progression
# • Rate of increase is steeper in the OFF state, showing more pronounced worsening when unmedicated

# 3. Late-Stage Volatility (Visits ~30–34)
# --------------------------------------
# • Both curves become more erratic near the end (especially Visit 34)
# • Likely causes:
#   - Small sample sizes (dropout attrition over time)
#   - Increased heterogeneity in late disease stages
#   - Data artifacts (Visit 34 shows dramatic jump in ON state)

# Key Insights
# -------------
# • Medication (ON state) consistently reduces symptom burden, but doesn't stop progression
# • Gap between ON and OFF widens slightly over time, suggesting increasing reliance on medication
# • End-point spikes should be examined closely for possible outliers or data sparsity

In [ ]:
# 3. Boxplots/violin plots: NP3TOT by visit and PDSTATE
plt.figure(figsize=(14, 6))
sns.boxplot(x='VISIT_NUM', y='NP3TOT', hue='PDSTATE', data=data, showfliers=False)
plt.title('Boxplot: NP3TOT by Visit and PDSTATE')
plt.xlabel('Visit Number')
plt.ylabel('NP3TOT')
plt.legend(title='PDSTATE')
plt.show()

plt.figure(figsize=(14, 6))
sns.violinplot(x='VISIT_NUM', y='NP3TOT', hue='PDSTATE', data=data, split=True, inner='quartile')
plt.title('Violin Plot: NP3TOT by Visit and PDSTATE')
plt.xlabel('Visit Number')
plt.ylabel('NP3TOT')
plt.legend(title='PDSTATE')
plt.show()

In [ ]:
# 5. Heatmap: Correlation matrix for key NP3 items
corr_matrix = data[key_items].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix: Key NP3 Items')
plt.show()

# Machine Learning

In [ ]:
DATA_DIR_2 = '/Users/larsheijnen/Thesis/data/motor/MDS-UPDRS_Part_III_21Mar2025.csv'
machine_learning_data = pd.read_csv(DATA_DIR_2)

In [ ]:
# Convert date columns to datetime where possible
for col in ['INFODT', 'EXAMDT', 'ORIG_ENTRY', 'LAST_UPDATE']:
    if col in machine_learning_data.columns:
        machine_learning_data[col] = pd.to_datetime(machine_learning_data[col], errors='coerce')

# Convert specific date columns
machine_learning_data['EXAMDT'] = pd.to_datetime(machine_learning_data['EXAMDT'], errors='coerce')
machine_learning_data['INFODT'] = pd.to_datetime(machine_learning_data['INFODT'], errors='coerce')

# Fallback: fill missing EXAMDT with INFODT
machine_learning_data['EXAMDT'] = machine_learning_data['EXAMDT'].fillna(machine_learning_data['INFODT'])

# Replace 101 values with NaN in NP3 columns
np3_cols = [col for col in machine_learning_data.columns if col.startswith('NP3') and col != 'NP3TOT']
machine_learning_data[np3_cols] = machine_learning_data[np3_cols].replace(101, np.nan)

#Check outliers or wrong values
np3_cols = [col for col in machine_learning_data.columns if col.startswith('NP3') and col != 'NP3TOT']

# Find out-of-range values
out_of_range = (machine_learning_data[np3_cols] < 0) | (machine_learning_data[np3_cols] > 4)
if out_of_range.any().any():
    print("Warning: Out-of-range NP3 scores detected!")
    display(machine_learning_data.loc[out_of_range.any(axis=1), ['PATNO', 'EXAMDT'] + np3_cols])
else:
    print("All NP3 scores within expected range (0–4). Ignored NP3TOT.")

# Drop rows where NP3TOT is NaN
machine_learning_data = machine_learning_data.dropna(subset=['NP3TOT'])
print(f"Remaining rows after dropping NaN NP3TOT: {len(machine_learning_data)}")


In [5]:
motor_features = [
    'PATNO', 'EVENT_ID', 'INFODT',  # Identifying information

    # MDS-UPDRS Part III scores
    'NP3SPCH', 'NP3FACXP',
    'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL',  # Rigidity
    'NP3FTAPR', 'NP3FTAPL',  # Finger tapping
    'NP3HMOVR', 'NP3HMOVL',  # Hand movements
    'NP3PRSPR', 'NP3PRSPL',  # Pronation-supination
    'NP3TTAPR', 'NP3TTAPL',  # Toe tapping
    'NP3LGAGR', 'NP3LGAGL',  # Leg agility
    'NP3RISNG',  # Arising from chair
    'NP3GAIT', 'NP3FRZGT',  # Gait and freezing
    'NP3PSTBL',  # Postural stability
    'NP3POSTR',  # Posture
    'NP3BRADY',  # Body bradykinesia
    'NP3PTRMR', 'NP3PTRML',  # Postural tremor
    'NP3KTRMR', 'NP3KTRML',  # Kinetic tremor
    'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON',  # Rest tremor
    'NP3TOT'  # Total score (needed for target creation)
]


In [ ]:
# Keep only relevant columns available in the dataset
motor_df = machine_learning_data[[col for col in motor_features if col in machine_learning_data.columns]].copy() # Use .copy() to avoid SettingWithCopyWarning
print(f"Initial motor assessment DataFrame shape: {motor_df.shape}")

In [ ]:
# Define the expected components of NP3TOT based on MDS-UPDRS guidelines
# These are the column names as they appear in your CSV/DataFrame
expected_np3_components = [
    'NP3SPCH',  # 3.1
    'NP3FACXP', # 3.2
    'NP3RIGN',  # 3.3a
    'NP3RIGRU', # 3.3b
    'NP3RIGLU', # 3.3c
    'NP3RIGRL', # 3.3d
    'NP3RIGLL', # 3.3e
    'NP3FTAPR', # 3.4a
    'NP3FTAPL', # 3.4b
    'NP3HMOVR', # 3.5a
    'NP3HMOVL', # 3.5b
    'NP3PRSPR', # 3.6a
    'NP3PRSPL', # 3.6b
    'NP3TTAPR', # 3.7a
    'NP3TTAPL', # 3.7b
    'NP3LGAGR', # 3.8a
    'NP3LGAGL', # 3.8b
    'NP3RISNG', # 3.9
    'NP3GAIT',  # 3.10
    'NP3FRZGT', # 3.11
    'NP3PSTBL', # 3.12
    'NP3POSTR', # 3.13
    'NP3BRADY', # 3.14
    'NP3PTRMR', # 3.15a (Postural Tremor Right Hand)
    'NP3PTRML', # 3.15b (Postural Tremor Left Hand)
    'NP3KTRMR', # 3.16a (Kinetic Tremor Right Hand)
    'NP3KTRML', # 3.16b (Kinetic Tremor Left Hand)
    'NP3RTARU', # 3.17a (Rest Tremor Right Upper Extremity)
    'NP3RTALU', # 3.17b (Rest Tremor Left Upper Extremity)
    'NP3RTARL', # 3.17c (Rest Tremor Right Lower Extremity)
    'NP3RTALL', # 3.17d (Rest Tremor Left Lower Extremity)
    'NP3RTALJ', # 3.17e (Rest Tremor Jaw)
    'NP3RTCON'  # 3.18 (Rest Tremor Constancy)
]

# motor_features is defined in a previous cell.
# Let's find which features in motor_features are part of the expected NP3 components
actual_np3_sum_components_in_motor_features = [col for col in motor_features if col in expected_np3_components]

print(f"Number of expected NP3 components: {len(expected_np3_components)}")
print(f"Expected NP3 components: {expected_np3_components}")
print("-" * 50)
print(f"Number of NP3 sum components found in your 'motor_features' list: {len(actual_np3_sum_components_in_motor_features)}")
print(f"Actual NP3 sum components from 'motor_features': {actual_np3_sum_components_in_motor_features}")
print("-" * 50)

# Verify the sum for a few rows in the original motor_df (before any dropping of NaNs for specific columns if that happens later)
# Make sure motor_df is loaded and NP3TOT is present
if 'motor_df' in locals() and 'NP3TOT' in motor_df.columns:
    # Calculate the sum of these components for each row
    motor_df['Calculated_NP3TOT'] = motor_df[actual_np3_sum_components_in_motor_features].sum(axis=1)

    # Compare with the original NP3TOT
    # We'll check the first 5 rows and a few random rows where NP3TOT is not 0 (if any)
    comparison_df = motor_df[['NP3TOT', 'Calculated_NP3TOT']].copy()
    comparison_df['Difference'] = comparison_df['NP3TOT'] - comparison_df['Calculated_NP3TOT']

    print("Comparison of NP3TOT with sum of its identified components (first 5 rows):")
    print(comparison_df.head())
    print("-" * 50)

    # Check if all differences are zero (or very close to zero, allowing for potential float precision issues if data was manipulated)
    if comparison_df['Difference'].abs().sum() < 1e-5: # A small tolerance for floating point arithmetic
        print("Verification successful: NP3TOT matches the sum of the identified components for all rows.")
    else:
        print("Verification FAILED: NP3TOT does NOT match the sum of the identified components for all rows.")
        print(f"Number of rows with mismatch: {len(comparison_df[comparison_df['Difference'].abs() > 1e-5])}")
        print("Rows with discrepancies:")
        print(comparison_df[comparison_df['Difference'].abs() > 1e-5].head())

    # It's good practice to drop the temporary column if you don't need it later
    # motor_df.drop(columns=['Calculated_NP3TOT'], inplace=True)
else:
    print("motor_df not found or NP3TOT column is missing. Cannot perform sum verification.")


In [ ]:
print("Missing values in motor_df BEFORE any dropna (immediately after creation):")
print(f"Shape of motor_df: {motor_df.shape}")
print("\nNaNs per column in motor_features + NP3TOT:")
nan_counts = motor_df[motor_features + ['NP3TOT']].isnull().sum()
print(nan_counts[nan_counts > 0]) # Only print columns that have NaNs

print(f"\nTotal rows with any NaN in motor_features: {motor_df[motor_features].isnull().any(axis=1).sum()}")
print(f"Total rows with NaN in NP3TOT: {motor_df['NP3TOT'].isnull().sum()}")

# Display a few rows that have NaNs in any of the motor_feature columns for context
if motor_df[motor_features].isnull().any(axis=1).sum() > 0:
    print("\nExample rows with NaNs in motor_features:")
    print(motor_df[motor_df[motor_features].isnull().any(axis=1)].head())

In [9]:
motor_df.sort_values(['PATNO', 'INFODT'], inplace=True)

motor_df['days_since_baseline'] = motor_df.groupby('PATNO')['INFODT'].transform(lambda x: (x - x.min()).dt.days)

# --- Create the Target Variable: NP3TOT of the *next* visit ---
motor_df['NP3TOT_next_visit'] = motor_df.groupby('PATNO')['NP3TOT'].shift(-1)

# --- Create Lag Features (Example: Previous NP3TOT) ---
# This demonstrates adding explicit time features. Can add more lags or differences.
motor_df['NP3TOT_lag1'] = motor_df.groupby('PATNO')['NP3TOT'].shift(1)
motor_df['NP3TOT_diff1'] = motor_df.groupby('PATNO')['NP3TOT'].diff(1)
# Add lag 2 features
motor_df['NP3TOT_lag2'] = motor_df.groupby('PATNO')['NP3TOT'].shift(2)
motor_df['NP3TOT_diff2'] = motor_df.groupby('PATNO')['NP3TOT'].diff(2)
# Add lag 3 features
motor_df['NP3TOT_lag3'] = motor_df.groupby('PATNO')['NP3TOT'].shift(3)
motor_df['NP3TOT_diff3'] = motor_df.groupby('PATNO')['NP3TOT'].diff(3)

motor_df['days_since_prev1'] = motor_df.groupby('PATNO')['INFODT'].diff(1).dt.days
motor_df['days_since_prev2'] = motor_df.groupby('PATNO')['INFODT'].diff(2).dt.days
motor_df['days_since_prev3'] = motor_df.groupby('PATNO')['INFODT'].diff(3).dt.days

In [ ]:
# Drop rows where the target ('NP3TOT_next_visit') is NaN (these are the last visits per patient)
# Also drop rows where lag features are NaN (first visits)
progression_df = motor_df.dropna(subset=[
    'NP3TOT_next_visit', 
    'NP3TOT_lag1', 'NP3TOT_diff1', 
    'NP3TOT_lag2', 'NP3TOT_diff2',
    'NP3TOT_lag3', 'NP3TOT_diff3',
    'days_since_prev1', 'days_since_prev2', 'days_since_prev3'
]).copy()

print(f"\nShape after creating target and lag features & dropping NaNs: {progression_df.shape}")

In [ ]:
# Original selection of NP3 items (current visit)
individual_np3_items = [col for col in progression_df.columns if 
                        col.startswith('NP3') and 
                        col not in ['NP3TOT', 'NP3TOT_next_visit'] and
                        not col.startswith('NP3TOT_lag') and 
                        not col.startswith('NP3TOT_diff')]

# Explicitly list all engineered features to ensure no overlap and correct selection
engineered_features = [
    'days_since_baseline',
    'NP3TOT_lag1', 'NP3TOT_diff1',
    'NP3TOT_lag2', 'NP3TOT_diff2',
    'NP3TOT_lag3', 'NP3TOT_diff3',
    'days_since_prev1', 'days_since_prev2', 'days_since_prev3'
]

feature_cols = individual_np3_items + engineered_features

# Ensure all listed engineered_features actually exist in progression_df.columns to avoid errors
feature_cols = [col for col in feature_cols if col in progression_df.columns]
# Remove duplicates just in case, and maintain order
from collections import OrderedDict
feature_cols = list(OrderedDict.fromkeys(feature_cols))


X = progression_df[feature_cols]
y = progression_df['NP3TOT_next_visit'] # y definition remains the same
groups = progression_df['PATNO'] # groups definition remains the same

print(f"\nCorrected Feature shape (X): {X.shape}")
print(f"Target shape (y): {y.shape}")
print(f"Group shape: {groups.shape}")
print("\nCorrected Features used:")
print(X.columns.tolist())
print(f"\nNumber of features: {len(X.columns.tolist())}")

In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
groups_train = groups.iloc[train_idx]
groups_test = groups.iloc[test_idx]

print(f"\nTrain shapes: X={X_train.shape}, y={y_train.shape}, groups={groups_train.shape}")
print(f"Test shapes: X={X_test.shape}, y={y_test.shape}, groups={groups_test.shape}")
print(f"Number of unique patients in train: {groups_train.nunique()}")
print(f"Number of unique patients in test: {groups_test.nunique()}")

# Verify no patient overlap (should be empty)
train_patients = set(groups_train.unique())
test_patients = set(groups_test.unique())
print(f"Overlap patients: {train_patients.intersection(test_patients)}")

In [ ]:
scaler = StandardScaler()

# Fit scaler ONLY on training data
scaler.fit(X_train)

# Transform both train and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

print("\nScaled training data head:")
X_train_scaled.head()

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "RandomForest Regressor": RandomForestRegressor(random_state=42, n_estimators=100), # Good baseline
    "GradientBoosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror'), # Specify objective for regression
    "KNeighbors": KNeighborsRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42, early_stopping=True) # Added early stopping
}

results_test = []

In [ ]:
# ## 7. Model Training and Evaluation (Single Split)

# Define models to train (Added RandomForest)
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "RandomForest Regressor": RandomForestRegressor(random_state=42, n_estimators=100),
    "GradientBoosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror'),
    "KNeighbors": KNeighborsRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42, early_stopping=True)
}

# Initialize results list
results_test = []

print("\n--- Evaluating on Test Set ---")
# Train and evaluate each model on the single train/test split
for name, model in models.items():
    print(f"Training {name}...")
    # --- MODIFICATION START ---
    # Convert DataFrames to NumPy arrays for fitting and prediction
    # Ensure y_train and y_test are also Series/arrays (which they should be)
    model.fit(X_train_scaled.values, y_train) # Use .values
    y_pred = model.predict(X_test_scaled.values) # Use .values
    # --- MODIFICATION END ---

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results_test.append({"Model": name, "MAE": mae, "RMSE": rmse, "R-squared": r2})
    print(f"{name}: MAE={mae:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}")

    # Plot actual vs predicted for the test set
    plt.figure(figsize=(6, 5))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.xlabel("Actual NP3TOT (Next Visit)")
    plt.ylabel("Predicted NP3TOT (Next Visit)")
    plt.title(f"Test Set: Actual vs Predicted ({name})\nR2={r2:.3f}")
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--') # Add y=x line
    plt.tight_layout()
    plt.show()

# Write test results to a CSV file
# csv_file_test = "/Users/larsheijnen/Thesis/Modelling/Motor-PPMI/results_motor/model_results_progression_test_lag3.csv"
# try:
#     with open(csv_file_test, mode='w', newline='') as file:
#         writer = csv.DictWriter(file, fieldnames=["Model", "MAE", "RMSE", "R-squared"])
#         writer.writeheader()
#         writer.writerows(results_test)
#     print(f"\nTest set results written to {csv_file_test}")
# except IOError as e:
#     print(f"Error writing file {csv_file_test}: {e}")

# Display test results as DataFrame
results_test_df = pd.DataFrame(results_test)
print("\nTest Set Performance Summary:")
print(results_test_df.round(3))

In [ ]:
# Initialize results list for CV
results_cv = []

# Define cross-validation strategy
cv = GroupKFold(n_splits=5) # Use GroupKFold


print("\n--- Evaluating with Cross-Validation (on Training Data) ---")

for name, model in models.items():
    print(f"Cross-validating {name}...")

    # --- MODIFICATION START ---
    # Get cross-validated predictions on the scaled training data (NumPy array)
    y_pred_cv = cross_val_predict(model, X_train_scaled.values, y_train, groups=groups_train, cv=cv) # Use .values
    # --- MODIFICATION END ---

    mae_cv = mean_absolute_error(y_train, y_pred_cv)
    rmse_cv = np.sqrt(mean_squared_error(y_train, y_pred_cv))
    r2_cv = r2_score(y_train, y_pred_cv)

    results_cv.append({"Model": name, "MAE": mae_cv, "RMSE": rmse_cv, "R-squared": r2_cv})
    print(f"{name}: MAE={mae_cv:.3f}, RMSE={rmse_cv:.3f}, R2={r2_cv:.3f}")

    # Plot actual vs predicted for CV
    plt.figure(figsize=(6, 5))
    plt.scatter(y_train, y_pred_cv, alpha=0.5)
    plt.xlabel("Actual NP3TOT (Next Visit)")
    plt.ylabel("Predicted NP3TOT (Next Visit)")
    plt.title(f"CV (Train Set): Actual vs Predicted ({name})\nR2={r2_cv:.3f}")
    min_val = min(y_train.min(), y_pred_cv.min())
    max_val = max(y_train.max(), y_pred_cv.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--') # Add y=x line
    plt.tight_layout()
    plt.show()


# Write CV results to a CSV file in the specified directory
# csv_file_cv = "/Users/larsheijnen/Thesis/Modelling/Motor-PPMI/results_motor/model_results_progression_test__cv_lag3.csv"
# try:
#     with open(csv_file_cv, mode='w', newline='') as file:
#         writer = csv.DictWriter(file, fieldnames=["Model", "MAE", "RMSE", "R-squared"])
#         writer.writeheader()
#         writer.writerows(results_cv)
#     print(f"\nCross-validation results written to {csv_file_cv}")
# except IOError as e:
#     print(f"Error writing file {csv_file_cv}: {e}")

# Display CV results as DataFrame
results_cv_df = pd.DataFrame(results_cv)
print("\nCross-Validation Performance Summary (on Training Data):")
print(results_cv_df.round(3))